In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_02a_why_sqrt5")' FastaiNotebook_02a_why_sqrt5

Installing packages:
	.package(path: "/home/jmd/workspace/ml/fastai/nbs/swift/FastaiNotebook_02a_why_sqrt5")
		FastaiNotebook_02a_why_sqrt5
With SwiftPM flags: []
Working in: /tmp/tmpd2s7kkzb/swift-install
/home/jmd/swift/usr/bin/swift-build: /home/jmd/anaconda3/lib/libcurl.so.4: no version information available (required by /home/jmd/swift/usr/lib/swift/linux/libFoundationNetworking.so)
[1/3] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/4] Merging module jupyterInstalledPackages
[3/3] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [2]:
//export
import Path
import TensorFlow

In [3]:
import FastaiNotebook_02a_why_sqrt5

In [4]:
// export
public typealias TI = Tensor<Int32>

In [5]:
var (xTrain,yTrain,xValid,yValid) = loadMNIST(path: Path.home/".fastai"/"data"/"mnist_tst", flat: true)

In [6]:
let trainMean = xTrain.mean()
let trainStd  = xTrain.standardDeviation()

In [7]:
xTrain = normalize(xTrain, mean: trainMean, std: trainStd)
xValid = normalize(xValid, mean: trainMean, std: trainStd)

In [8]:
let (n,m) = (xTrain.shape[0],xTrain.shape[1])
let c = yTrain.max().scalarized()+1
print(n,m,c)

60000 784 10


In [9]:
let nHid = 50

In [10]:
public struct MyModel: Layer {
    public var layer1: FADense<Float>
    public var layer2: FADense<Float>
    
    public init(nIn: Int, nHid: Int, nOut: Int){
        layer1 = FADense(nIn, nHid, activation: relu)
        layer2 = FADense(nHid, nOut)
    }
    
    @differentiable
    public func callAsFunction(_ input: TF) -> TF {
        return input.sequenced(through: layer1, layer2)
    }
}

In [11]:
var model = MyModel(nIn: m, nHid: nHid, nOut: Int(c))

In [12]:
let pred = model(xTrain)

In [13]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    let exped = exp(activations) 
    return log(exped / exped.sum(alongAxes: -1))
}

In [14]:
let smPred = logSoftmax(pred)

In [15]:
yTrain[0..<3]

[5, 0, 4]


In [16]:
(smPred[0][5],smPred[1][0],smPred[2][4])

▿ 3 elements
  - .0 : -3.6093674
  - .1 : -3.1815867
  - .2 : -2.280293


In [17]:
func nll<Scalar>(_ input: Tensor<Scalar>, _ target :TI) -> Tensor<Scalar> 
    where Scalar:TensorFlowFloatingPoint{
        let idx: TI = _Raw.range(start: Tensor(0), limit: Tensor(numericCast(target.shape[0])), delta: Tensor(1))
        let indices = _Raw.concat(concatDim: Tensor(1), [idx.expandingShape(at: 1), target.expandingShape(at: 1)])
        let losses = _Raw.gatherNd(params: input, indices: indices)
        return -losses.mean()
    }

In [18]:
nll(smPred, yTrain)

2.5352411


In [19]:
time(repeating: 100){ let _ = nll(smPred, yTrain) }

average: 1.3821034900000007 ms,   min: 0.708999 ms,   max: 3.365624 ms


In [20]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    return activations - log(exp(activations).sum(alongAxes: -1))
}

In [21]:
let smPred = logSoftmax(pred)

In [22]:
nll(smPred, yTrain)

2.5352411


In [23]:
smPred.max(alongAxes: -1).shape

▿ [60000, 1]
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 1


In [24]:
func logSumExp<Scalar>(_ x: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    let m = x.max(alongAxes: -1)
    return m + log(exp(x-m).sum(alongAxes: -1))
}

In [25]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    return activations - logSumExp(activations)
}

In [26]:
let smPred = logSoftmax(pred)

In [27]:
nll(smPred, yTrain)

2.5352411


In [28]:
let loss = softmaxCrossEntropy(logits: pred, labels: yTrain)
loss

2.5352411


In [29]:
time(repeating: 100){ _ = nll(logSoftmax(pred), yTrain)}

average: 1.2442005399999998 ms,   min: 0.482056 ms,   max: 6.137875 ms


In [30]:
time(repeating: 100){ _ = softmaxCrossEntropy(logits: pred, labels: yTrain)}

average: 0.053128540000000016 ms,   min: 0.052441 ms,   max: 0.060195 ms


In [31]:
// export
public func accuracy(_ output: TF, _ target: TI) -> TF{
    let corrects = TF(output.argmax(squeezingAxis: 1) .== target)
    return corrects.mean()
}

In [32]:
print(accuracy(pred, yTrain))

0.09791667


In [33]:
let bs=64                     // batch size
let xb = xTrain[0..<bs]       // a mini-batch from x
let preds = model(xb)         // predictions
print(preds[0], preds.shape)

[-0.26924807,   0.7771155,   1.5648208, -0.03286986,  0.06529951, -0.88017595,    0.882473,
 -0.00849247,  -0.4475536,  0.09719473] [64, 10]


In [34]:
let yb = yTrain[0..<bs]
let loss = softmaxCrossEntropy(logits: preds, labels: yb)

In [35]:
print(accuracy(preds, yb))

0.078125


In [36]:
let lr:Float = 0.5   // learning rate
let epochs = 1       // how many epochs to train for

In [37]:
let (loss, grads) = valueWithGradient(at: model) { model -> TF in
    let preds = model(xb)
    return softmaxCrossEntropy(logits: preds, labels: yb)
}

In [38]:
for epoch in 1 ... epochs {
    for i in 0 ..< (n-1)/bs {
        let startIdx = i * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = valueWithGradient(at: model) {
            softmaxCrossEntropy(logits: $0(xb), labels: yb)
        }
        model.layer1.weight -= lr * grads.layer1.weight
        model.layer1.bias   -= lr * grads.layer1.bias
        model.layer2.weight -= lr * grads.layer2.weight
        model.layer2.bias   -= lr * grads.layer2.bias
    }
}

In [39]:
let preds = model(xValid)
accuracy(preds, yValid)

0.8957


In [40]:
for epoch in 1 ... epochs {
    for i in 0 ..< (n-1)/bs {
        let startIdx = i * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = valueWithGradient(at: model) {
            softmaxCrossEntropy(logits: $0(xb), labels: yb)
        }
        model.move(along: grads.scaled(by: -lr))
    }
}

In [41]:
let optimizer = SGD(for: model, learningRate: lr)

In [42]:
//export
public func batchedRanges(start:Int, end:Int, bs:Int) -> UnfoldSequence<Range<Int>,Int>
{
  return sequence(state: start) { (batchStart) -> Range<Int>? in
    let remaining = end - batchStart
    guard remaining > 0 else { return nil}
    let currentBs = min(bs,remaining)
    let batchEnd = batchStart.advanced(by: currentBs)
    defer {  batchStart = batchEnd  }
    return batchStart ..< batchEnd
  }
}

In [43]:
for epoch in 1 ... epochs{
    for b in batchedRanges(start: 0, end: n, bs: bs) {
        let (xb,yb) = (xTrain[b],yTrain[b])
        let (loss, grads) = valueWithGradient(at: model) {
            softmaxCrossEntropy(logits: $0(xb), labels: yb)
        }
        optimizer.update(&model, along: grads)
    }
}

In [44]:
// export
public struct DataBatch<Inputs: Differentiable & TensorGroup, Labels: TensorGroup>: TensorGroup {
    public var xb: Inputs
    public var yb: Labels
    
    public init(xb: Inputs, yb: Labels){ (self.xb,self.yb) = (xb,yb) }
    
    public var _tensorHandles: [_AnyTensorHandle] {
        xb._tensorHandles + yb._tensorHandles
    }
    
    public init<C: RandomAccessCollection>(_handles: C) where C.Element: _AnyTensorHandle {
        let xStart = _handles.startIndex
        let xEnd = _handles.index(
            xStart, offsetBy: Int(Inputs._tensorHandleCount))
        self.xb = Inputs.init(_handles: _handles[xStart..<xEnd])
        self.yb = Labels.init(_handles: _handles[xEnd..<_handles.endIndex])
    }
}

In [45]:
let trainDs = Dataset(elements:DataBatch(xb:xTrain, yb:yTrain)).batched(bs)

In [46]:
for epoch in 1...epochs{
    for batch in trainDs {
        let (loss, grads) = valueWithGradient(at: model) {
            softmaxCrossEntropy(logits: $0(xb), labels: yb)
        }
        optimizer.update(&model, along: grads)
    }
}

In [47]:
public func train<Opt: Optimizer, Label:TensorGroup>(
    _ model: inout Opt.Model,
    on ds: Dataset<DataBatch<Opt.Model.Input, Label>>,
    using opt: inout Opt,
    lossFunc: @escaping @differentiable (Opt.Model.Output, @noDerivative Label) -> Tensor<Opt.Scalar>
) where Opt.Model: Layer,
        Opt.Model.Input: TensorGroup,
        Opt.Scalar: TensorFlowFloatingPoint
{
    for batch in ds {
        let (loss, 𝛁model) = valueWithGradient(at: model) {
            lossFunc($0(batch.xb), batch.yb)
        }
        opt.update(&model, along: 𝛁model)
    }
}

In [48]:
var model = MyModel(nIn: m, nHid: nHid, nOut: Int(c))
var optimizer = SGD(for: model, learningRate: lr)

In [49]:
//export
@differentiable(wrt: logits)
public func crossEntropy(_ logits: TF, _ labels: TI) -> TF {
    return softmaxCrossEntropy(logits: logits, labels: labels)
}

In [50]:
train(&model, on: trainDs, using: &optimizer, lossFunc: crossEntropy)

In [51]:
let preds = model(xValid)
accuracy(preds, yValid)

0.894


In [52]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"03_minibatch_training.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
